# COMP30760 Assignment 2 - Task 1

For this assignment I decided to compare and analyse historical stock price data of 4 automobile companies using time series analysis. The companies are:-
- Honda Motor Company 
- Toyota Motor Corporation
- Ford Motor Company
- General Motors Company


In [257]:
import json, requests, urllib
from pathlib import Path
from datetime import datetime
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt

In [258]:
#URL of the ucd website from where we will collect the data
URL = "http://mlg.ucd.ie/modules/COMP30760/stocks/data-" 

#List of Automobile companies well collect data of
Companies = ["Honda Motor Company", "Toyota Motor Corporation", "Ford Motor Company" , "General Motors Company"]

#Companies and their stock market name
Stock_Names = { "Honda Motor Company" : "hmc", "Toyota Motor Corporation" : "tm" , "Ford Motor Company" : "f", "General Motors Company" : "gm" }

#years of data available
years = ["2019", "2020", "2021"]



In [259]:
# directory for raw data storage
dir_raw = Path("Assignment2_data")
# make sure it exists
dir_raw.mkdir(parents=True, exist_ok=True)

## Data Collection
below is the function we will use to fetch the HTML data from the UCD website.

In [260]:
def fetch() :
    df_meta = {}
    for company in Companies:
        # construct the url
        url = URL
        url += Stock_Names[company]
        url += ".html"
        print("Fetching %s" % url)
        #fetch data
        df_meta[company]= pd.read_html(url)
    return df_meta

In [261]:
#dataframe to store the raw data of companies 
df_Companies = {}

#calling the fetch function
df_Companies = fetch()


Fetching http://mlg.ucd.ie/modules/COMP30760/stocks/data-hmc.html
Fetching http://mlg.ucd.ie/modules/COMP30760/stocks/data-tm.html
Fetching http://mlg.ucd.ie/modules/COMP30760/stocks/data-f.html
Fetching http://mlg.ucd.ie/modules/COMP30760/stocks/data-gm.html


## Data Cleaning and Preprocessing

In [272]:
df_Companies[Companies[0]]

[   Stock  Year  Month  Day       Open       High        Low      Close
 0    HMC  2019     10    1  26.549999  26.580000  26.049999  26.150000
 1    HMC  2019     10    2  25.850000  25.889999  25.620001  25.709999
 2    HMC  2019     10    3  25.540001  25.730000  25.370001  25.730000
 3    HMC  2019     10    4  25.760000  25.950001  25.719999  25.950001
 4    HMC  2019     10    7        NaN        NaN        NaN        NaN
 ..   ...   ...    ...  ...        ...        ...        ...        ...
 59   HMC  2019     12   24  28.610001  28.680000  28.549999  28.660000
 60   HMC  2019     12   26  28.750000  28.780001  28.700001  28.760000
 61   HMC  2019     12   27  28.740000  28.740000  28.549999  28.580000
 62   HMC  2019     12   30  28.330000  28.469999  28.240000  28.270000
 63   HMC  2019     12   31  28.240000  28.340000  28.180000  28.309999
 
 [64 rows x 8 columns],
     Stock  Year  Month  Day       Open       High        Low      Close
 0     HMC  2020      1    2  28.6000

the fetched data for each company is divided by years. To join the years together we will use the function concat().

In [287]:
df_Companies_concat = {}

for company in Companies:  
    df_Companies_concat[company] = pd.concat(df_Companies[company])

df_Companies_concat[Companies[0]]


,Stock,Year,Month,Day,Open,High,Low,Close
0,HMC,2019,10,1,26.549999,26.580000,26.049999,26.150000
1,HMC,2019,10,2,25.850000,25.889999,25.620001,25.709999
2,HMC,2019,10,3,25.540001,25.730000,25.370001,25.730000
3,HMC,2019,10,4,25.760000,25.950001,25.719999,25.950001
4,HMC,2019,10,7,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
183,HMC,2021,9,24,30.820000,31.000000,30.760000,30.950001
184,HMC,2021,9,27,31.150000,31.500000,31.070000,31.469999
185,HMC,2021,9,28,NaN,NaN,NaN,NaN
186,HMC,2021,9,29,31.500000,31.750000,31.360001,31.600000


now that we have merged the years, lets handle the missing data. as we can see in the table above, there are some missing values (NAN) in the data. below we check how many missing values are there in each companies dataframe.

In [274]:
for company in Companies:
    print("-- %s" % company)
    missing_counts = df_Companies_concat[company].isnull().sum()
    # which coludmns have missing values?
    missing_counts = missing_counts[missing_counts>0]
    if len(missing_counts) == 0:
        print("No missing values found")
    else:
        print(missing_counts)

-- Honda Motor Company
Open     16
High     16
Low      16
Close    28
dtype: int64
-- Toyota Motor Corporation
Open      9
High      9
Low       9
Close    34
dtype: int64
-- Ford Motor Company
Open     38
High     38
Low      38
Close    38
dtype: int64
-- General Motors Company
Open     32
High     32
Low      32
Close    32
dtype: int64


We will replace the NAN values using the backward fill method which replace the null value with the next available value in the column. I also found in the data that there were some partitions made between the years using the symbol "-". We will remove those partitions below using drop().

In [278]:
# df_Companies_final[Companies[3]].iloc[173,2]
for company in Companies:
    df_Companies_concat[company].fillna(method="bfill", inplace = True)
    df_Companies_concat[company].drop(df_Companies_concat[company][df_Companies_concat[company]['High']== '—'].index, inplace=True)


Now we will again check if there are any null values left 

In [279]:
df_Companies_final[Companies[3]].iloc[193,1]



for company in Companies:
    print("-- %s" % company)
    missing_counts = df_Companies_concat[company].isnull().sum()
    # which coludmns have missing values?
    missing_counts = missing_counts[missing_counts>0]
    if len(missing_counts) == 0:
        print("No missing values found")
    else:
        print(missing_counts)

-- Honda Motor Company
No missing values found
-- Toyota Motor Corporation
No missing values found
-- Ford Motor Company
No missing values found
-- General Motors Company
No missing values found


Some of the company's data have month written in alphabetic form. we will change them into the numeric form for future use. We will do this using the apply() function.

In [283]:
df_Companies_concat[Companies[3]]

,Stock,Day,Month,Year,Open,High,Low,Close
0,GM,01,10,2019,37.470001,37.480000,36.029999,36.110001
1,GM,02,10,2019,35.770000,35.770000,34.410000,34.680000
2,GM,03,10,2019,34.509998,34.990002,34.160000,34.980000
3,GM,04,10,2019,35.000000,35.189999,34.580002,34.910000
4,GM,07,10,2019,34.599998,35.110001,34.299999,34.750000
...,...,...,...,...,...,...,...,...
189,GM,24,9,2021,51.880001,52.630001,51.720001,52.230000
190,GM,27,9,2021,52.410000,53.570000,52.380001,53.240002
191,GM,28,9,2021,53.770000,54.209999,52.700001,52.849998
192,GM,29,9,2021,53.299999,53.360001,52.529999,52.930000


In [280]:
df_Companies_final[Companies[3]]
import calendar

month_name_abbr = {"Jan": 1,
              "Feb": 2,
              "Mar": 3,
              "Apr": 4,
              "May": 5,
              "Jun": 6,
              "Jul": 7,
              "Aug": 8,
              "Sep": 9,
              "Oct": 10,
              "Nov": 11,
              "Dec": 12}

month_name = {"January": 1,
              "February": 2,
              "March": 3,
              "April": 4,
              "May": 5,
              "June": 6,
              "July": 7,
              "August": 8,
              "September": 9,
              "October": 10,
              "November": 11,
              "December": 12}

df_Companies_concat[Companies[2]]['Month'] = df_Companies_concat[Companies[2]]['Month'].apply(lambda x: month_name[x])
df_Companies_concat[Companies[3]]['Month'] = df_Companies_concat[Companies[3]]['Month'].apply(lambda x: month_name_abbr[x])

Lets check if the months have been replaced.

In [282]:
df_Companies_concat[Companies[3]]

,Stock,Day,Month,Year,Open,High,Low,Close
0,GM,01,10,2019,37.470001,37.480000,36.029999,36.110001
1,GM,02,10,2019,35.770000,35.770000,34.410000,34.680000
2,GM,03,10,2019,34.509998,34.990002,34.160000,34.980000
3,GM,04,10,2019,35.000000,35.189999,34.580002,34.910000
4,GM,07,10,2019,34.599998,35.110001,34.299999,34.750000
...,...,...,...,...,...,...,...,...
189,GM,24,9,2021,51.880001,52.630001,51.720001,52.230000
190,GM,27,9,2021,52.410000,53.570000,52.380001,53.240002
191,GM,28,9,2021,53.770000,54.209999,52.700001,52.849998
192,GM,29,9,2021,53.299999,53.360001,52.529999,52.930000


Now that we have cleaned the data, we will save it in a directory.

In [284]:
for company in Companies:
    filename = "%s.csv" % (company)
    df_Companies_concat[company].to_csv("/Users/sagarmahajan/Assignment2_data/%s" % filename)
    print("writing data for %s" % (company))

writing data for Honda Motor Company
writing data for Toyota Motor Corporation
writing data for Ford Motor Company
writing data for General Motors Company
